In [ ]:
##%overwritefile
##%file:src/compile_with_perlc.py
##%noruncode
    def compile_with_perlc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None,magics=None):
        outfile=binary_filename
        orig_cflags=cflags
        orig_ldflags=ldflags
        index=0
        for s in cflags:
            if s.startswith('--outFile'):
                if(len(s)>9):
                    outfile=s[9:]
                    del cflags[index]
                else:
                    outfile=cflags[cflags.index('--outFile')+1]
                    if outfile.startswith('-'):
                        outfile=binary_filename
                    del cflags[cflags.index('--outFile')+1]
                    del cflags[cflags.index('--outFile')]
            binary_filename=outfile
            index+=1
        args=[]
        if magics!=None and len(self.addkey2dict(magics,'ccompiler'))>0:
            args = magics['ccompiler'] + orig_cflags +[source_filename] + orig_ldflags
        else:
            tsccmd=['tsc']
            if self.sys=="Windows":
                tsccmd=['cmd','/c','tsc']
            args = tsccmd+[source_filename] + ['--outFile', binary_filename]+ cflags  + ldflags
        # self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename,args


In [ ]:
##%overwritefile
##%file:src/c_exec_perlc_.py
##%noruncode
    def _exec_perlc_(self,source_filename,magics):
        self._logln('Generating executable file')
        with self.new_temp_file(suffix='.php') as binary_file:
            p,outfile,tsccmd = self.compile_with_perlc(
                source_filename, 
                binary_file.name,
                self.get_magicsSvalue(magics,'cflags'),
                self.get_magicsSvalue(magics,'ldflags'),
                self.get_magicsbykey(magics,'env'),
                magics=magics)
            returncode=p.wait_end(magics)
            p.write_contents()
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if returncode != 0:  # Compilation failed
                self._logln(' '.join((str(s) for s in tsccmd))+"\n",3)
                self._logln("TSC compiler exited with code {}, the executable will not be executed".format(returncode),3)

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name
